In [1]:
!pip install tensorflow_text beir hnswlib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from beir import util, LoggingHandler
from beir.retrieval import models
from beir.datasets.data_loader import GenericDataLoader
from beir.retrieval.evaluation import EvaluateRetrieval
from beir.retrieval.search.dense import DenseRetrievalExactSearch as DRES

import logging
import pathlib, os
import numpy as np
import time
import hnswlib

#### Just some code to print debug information to stdout
logging.basicConfig(
    format="%(asctime)s - %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
    level=logging.INFO,
    handlers=[LoggingHandler()],
)
#### /print debug information to stdout

#### Download scifact.zip dataset and unzip the dataset
dataset = "dbpedia-entity"
url = (
    "https://public.ukp.informatik.tu-darmstadt.de/thakur/BEIR/datasets/{}.zip".format(
        dataset
    )
)
out_dir = os.path.join(pathlib.Path(".").parent.absolute(), "datasets")
data_path = util.download_and_unzip(url, out_dir)

#### Provide the data_path where scifact has been downloaded and unzipped
corpus, queries, qrels = GenericDataLoader(data_folder=data_path).load(split="test")

/usr/local/lib/python3.7/dist-packages/beir/util.py:2: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


  0%|          | 0/4635922 [00:00<?, ?it/s]

`corpus`: un dictionnaire contenant en `clé` le titre de l'oeuvre et en `value` un autre dictionnaire contenant une `clé` "titre" et une deuxième `clé` "texte" avec le contenu.

`queries`: un dictionnaire contennant en value des recherche par mots clés pour rechercher des texte de mannière générique

`qrels`: un dictionnaire avec les même `clés` que `queries` où chacune d'elles aura comme valeur un autre dictionnaire avec des `clés` présentes dans `corpus` est donnant un niveau de raprochement entre le corpus et la querie 0 pour aucune ressemblance sinon 1 ou 2

In [3]:
# We need to remove the 0 values from qrels since it is not relevant
new_q = dict()
for item in qrels.items():
    new_q[item[0]] = {x[0]: x[1] for x in item[1].items() if x[1] != 0}
qrels = new_q

In [4]:
# Create a dict with all relevant text to queries
s = {y for x in qrels.values() for y in x.keys()}
res = {x: corpus[x]["text"] for x in s}

In [5]:
# Add random queries to this dict
np.random.seed(0)
res.update(
    {x: corpus[x]["text"] for x in np.random.choice(list(corpus), 100000, False)}
)

In [6]:
# Separate the dict between keys and values
little_corpus_keys = [x for x in res.keys()]
little_corpus = [x for x in res.values()]

As said in [the subject and explain in the doc](https://www.sbert.net/examples/applications/semantic-search/README.html#symmetric-vs-asymmetric-semantic-search).
Since we have small query and big text with it we decide to choose Asymmetric Semantic Search to be more efficient.


Once we know we want assymetric model, to do queries faster  we prefer to normalize the projection in the embedding part so we can do only dot product for query instead of cosine similarity. We will then choose model tuned for dot-product part in the this [table](https://www.sbert.net/docs/pretrained-models/msmarco-v3.html). We choose msmarco-distilbert-base-tas-b since he has the best perf overall in his category.



In [7]:
from sentence_transformers import SentenceTransformer, util

# Choose a model and load it
model = SentenceTransformer('msmarco-distilbert-base-tas-b')

In [8]:
# Sentences are encoded by calling model.encode()
embeddings = model.encode(little_corpus)

In [9]:
# Encode a queries and find the best result to see if it's working a little
querie_embedding = model.encode(queries["INEX_LD-2009022"])
found = np.argmax(util.cos_sim(querie_embedding, embeddings))
print("Similarity:", found)
little_corpus[found], queries["INEX_LD-2009022"]

Similarity: tensor(14284)


('Sichuan cuisine, Szechwan cuisine, or Szechuan cuisine (/ˈsɛʃwɒn/ or /ˈsɛtʃwɒn/; Chinese: 四川菜; pinyin: Sìchuān cài or Chinese: 川菜; pinyin: Chuān cài) is a style of Chinese cuisine originating from Sichuan province in southwestern China. It has bold flavours, particularly the pungency and spiciness resulting from liberal use of garlic and chili peppers, as well as the unique flavor of the Sichuan pepper.',
 'Szechwan dish food cuisine')

In [10]:
# Separates the dict between keys and values so it's easier to iterate
list_queries_keys = [x for x in queries.keys()]
list_queries = [x for x in queries.values()]

In [11]:
# Compute all queries to find the best text associated
start_time = time.time()

queries_embedding = model.encode(list_queries)


res_search = util.semantic_search(
    queries_embedding, embeddings, top_k=100, score_function=util.dot_score
)

end_time = time.time()
print(
    "Average time for a query:", (end_time - start_time) / len(list_queries), "seconds"
)

Average time for a query: 0.004452493190765381 seconds


In [12]:
def precision_k(retrieved: list, relevant: dict, k: int = 100) -> float:
    """
    Calculate for a querie defined before the precision of answer of the
    model return the precision (0 if the last answer is not in relevant)

    Parameters
    ----------
    retrieved : list
        list of text choose to answer the querie
    relevant : dict
        dict containing all relevant text for the querie
    k : int, optional
        size of selected queries for precision (default is 100)
        have to be smaller or equal of len(retrieved)

    Returns
    -------
    float
    """
    n_retrieve = k
    n_number = len(retrieved[:k] & relevant.keys())
    n_relevant = len(relevant)
    return (n_number / n_retrieve) * int(retrieved[k] in relevant)

In [13]:
r = 0
for i in range(len(res_search)):
    # Find for a query all corpus keys
    retrieved = [little_corpus_keys[x["corpus_id"]] for x in res_search[i]]
    # Find all relevant for a query
    relevant = qrels[list_queries_keys[i]]
    ap = 0
    for i in range(1, 100):
        ap += precision_k(retrieved, relevant, i)
    r += ap / len(relevant)
print("MAP@100:", r / len(res_search))

MAP@100: 0.4551685596506889


In [14]:
# Set all parameters
embedding_size = 768    #Size of embeddings
top_k_hits = 100        #Output k hits
index = hnswlib.Index(space = 'cosine', dim = embedding_size) # We set the space as cosine so we do not have to 

In [15]:
# Initialize index with every embedded text
index.init_index(max_elements = len(embeddings), ef_construction = 40, M = 64)
index.add_items(embeddings, list(range(len(embeddings))))

In [16]:
# Compute for every query
corpus_ids, distances = index.knn_query(queries_embedding, k=top_k_hits)

In [17]:
r = 0
# 0.4055050893647521
for i in range(len(corpus_ids)):
    # Find for a query all corpus keys
    hits = [
        {"corpus_id": id, "score": 1 - score}
        for id, score in zip(corpus_ids[i], distances[i])
    ]
    hits = sorted(hits, key=lambda x: x["score"], reverse=True)
    retrieved = [little_corpus_keys[x["corpus_id"]] for x in hits]

    # Find all relevant for a query
    relevant = qrels[list_queries_keys[i]]
    ap = 0
    limit = 100 if len(retrieved) > 100 else len(retrieved)
    for i in range(1, limit):
        ap += precision_k(retrieved, relevant, i)
    r += ap / len(relevant)
print("MAP@100:", r / len(res_search))

MAP@100: 0.40341242514412395
